In [ ]:
!pip install torch==2.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found ex

In [ ]:
import torch
print(torch.__version__)

2.5.1+cu124


In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.1+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00


# HeteroGNN

In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch_geometric.nn import to_hetero
from torch_geometric.datasets import IMDB
from torch_geometric.loader import NeighborLoader, DataLoader
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv, LightGCN
from torch.optim import Adam
from models import GraphSAGE
from model_utils import train_classification
from lightgcn_recommendation import train

In [ ]:
data = IMDB('IMDB')

Extracting IMDB/raw/IMDB_processed.zip
Processing...
Done!


In [ ]:
dataset = data[0]

In [ ]:
dataset

HeteroData(
  movie={
    x=[4278, 3066],
    y=[4278],
    train_mask=[4278],
    val_mask=[4278],
    test_mask=[4278],
  },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={ edge_index=[2, 12828] },
  (director, to, movie)={ edge_index=[2, 4278] },
  (actor, to, movie)={ edge_index=[2, 12828] }
)

In [ ]:
base_model = GraphSAGE(num_classes=3)

In [ ]:
base_model

GraphSAGE(
  (conv1): SAGEConv((-1, -1), 64, aggr=mean)
  (conv2): SAGEConv((-1, -1), 128, aggr=mean)
  (conv3): SAGEConv((-1, -1), 64, aggr=mean)
  (linear1): Linear(64, 3, bias=True)
)

In [ ]:
dataset.metadata()

(['movie', 'director', 'actor'],
 [('movie', 'to', 'director'),
  ('movie', 'to', 'actor'),
  ('director', 'to', 'movie'),
  ('actor', 'to', 'movie')])

In [ ]:
model = to_hetero(base_model, dataset.metadata(), aggr='sum')

/usr/local/lib/python3.11/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "
/usr/local/lib/python3.11/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout_1' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "
/usr/local/lib/python3.11/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout_2' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encaps

In [ ]:
model

GraphModule(
  (conv1): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 64, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 64, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (conv2): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 128, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 128, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 128, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 128, aggr=mean)
  )
  (conv3): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 64, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 64, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (linear1): ModuleDict(
    (movie): Linear(64, 3, bias=True)
    (director): Linear(64, 3, bias=True)
    (actor): Linear(64, 3, bias=True)
  )
)

In [ ]:
dataset['movie'].train_mask

tensor([False, False, False,  ..., False, False, False])

In [ ]:
train_input_nodes = ('movie', dataset['movie'].train_mask)
train_loader = NeighborLoader(dataset, num_neighbors=[3, 3],
                              shuffle=True, input_nodes=train_input_nodes,
                              batch_size=128)

In [ ]:
val_input_nodes = ('movie', dataset['movie'].val_mask)
val_loader = NeighborLoader(dataset, num_neighbors=[3, 3],
                            shuffle=False, input_nodes=val_input_nodes,
                            batch_size=128)

In [ ]:
next(iter(train_loader))

HeteroData(
  movie={
    x=[792, 3066],
    y=[792],
    train_mask=[792],
    val_mask=[792],
    test_mask=[792],
    n_id=[792],
    num_sampled_nodes=[3],
    input_id=[128],
    batch_size=128,
  },
  director={
    x=[125, 3066],
    n_id=[125],
    num_sampled_nodes=[3],
  },
  actor={
    x=[357, 3066],
    n_id=[357],
    num_sampled_nodes=[3],
  },
  (movie, to, director)={
    edge_index=[2, 269],
    e_id=[269],
    num_sampled_edges=[2],
  },
  (movie, to, actor)={
    edge_index=[2, 807],
    e_id=[807],
    num_sampled_edges=[2],
  },
  (director, to, movie)={
    edge_index=[2, 128],
    e_id=[128],
    num_sampled_edges=[2],
  },
  (actor, to, movie)={
    edge_index=[2, 384],
    e_id=[384],
    num_sampled_edges=[2],
  }
)

In [ ]:
optimizer = Adam(model.parameters(), lr=0.0001)

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
train_classification(model=model,
                     train_loader=train_loader,
                     val_loader=val_loader,
                     optimizer=optimizer,
                     criterion=criterion,
                     epochs=50)

Epoch: 000, Step: 000, Loss: 1.0168
Epoch: 000, Step: 001, Loss: 1.0104
Epoch: 000, Step: 002, Loss: 0.9763
Epoch: 000, Step: 003, Loss: 0.9771
Epoch: 000, Step: 000, Val Loss: 1.0880
Epoch: 000, Step: 001, Val Loss: 1.0288
Epoch: 000, Step: 002, Val Loss: 1.0182
Epoch: 000, Step: 003, Val Loss: 0.9799
Epoch: 001, Step: 000, Loss: 0.9610
Epoch: 001, Step: 001, Loss: 1.0075
Epoch: 001, Step: 002, Loss: 0.9734
Epoch: 001, Step: 003, Loss: 0.9690
Epoch: 001, Step: 000, Val Loss: 1.0849
Epoch: 001, Step: 001, Val Loss: 1.0324
Epoch: 001, Step: 002, Val Loss: 1.0241
Epoch: 001, Step: 003, Val Loss: 0.9257
Epoch: 002, Step: 000, Loss: 0.9360
Epoch: 002, Step: 001, Loss: 0.9724
Epoch: 002, Step: 002, Loss: 0.9670
Epoch: 002, Step: 003, Loss: 0.9526
Epoch: 002, Step: 000, Val Loss: 1.0729
Epoch: 002, Step: 001, Val Loss: 1.0231
Epoch: 002, Step: 002, Val Loss: 1.0002
Epoch: 002, Step: 003, Val Loss: 0.9527
Epoch: 003, Step: 000, Loss: 0.9418
Epoch: 003, Step: 001, Loss: 0.9484
Epoch: 003, Step

# LightGCN

In [ ]:
dataset

HeteroData(
  movie={
    x=[4278, 3066],
    y=[4278],
    train_mask=[4278],
    val_mask=[4278],
    test_mask=[4278],
  },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={ edge_index=[2, 12828] },
  (director, to, movie)={ edge_index=[2, 4278] },
  (actor, to, movie)={ edge_index=[2, 12828] }
)

In [ ]:
dataset2 = dataset.to_homogeneous()

In [ ]:
dataset2

Data(edge_index=[2, 34212], x=[11616, 3066], y=[11616], train_mask=[11616], val_mask=[11616], test_mask=[11616], node_type=[11616], edge_type=[34212])

In [ ]:
dataset2.num_nodes

11616

In [ ]:
model_lightgcn = LightGCN(num_nodes=dataset2.num_nodes,
                          embedding_dim=128,
                          num_layers=1)

In [ ]:
model_lightgcn

LightGCN(11616, 128, num_layers=1)

In [ ]:
dataset2.edge_index.size(1)

34212

In [ ]:
data_loader = DataLoader(range(dataset2.edge_index.size(1)),
                         shuffle=True,
                         batch_size=16)

In [ ]:
optimizer2 = Adam(model_lightgcn.parameters(), lr=0.0001)

In [ ]:
num_actor = dataset['actor'].num_nodes
num_actor

5257

In [ ]:
num_movie = dataset['movie'].num_nodes
num_movie

4278

In [ ]:
train(dataset=dataset2,
      train_loader=data_loader,
      model=model_lightgcn,
      optimizer=optimizer2,
      num_actors=num_actor,
      num_movies=num_movie,
      epochs=50)

Epoch: 000, Loss: 0.6921
Epoch: 000, Loss: 0.6916
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6916
Epoch: 000, Loss: 0.6916
Epoch: 000, Loss: 0.6916
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6914
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915
Epoch: 000, Loss: 0.6915


KeyboardInterrupt: 